In [ ]:
!pip -q install transformers pandas

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import pandas as pd

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("thanaphatt1/WangchanBERTa-LST20")
model = AutoModelForTokenClassification.from_pretrained("thanaphatt1/WangchanBERTa-LST20")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/419M [00:00<?, ?B/s]

In [ ]:
with open("/content/ws_test.txt", "r", encoding="utf-8") as f:
    text = f.read().strip()

In [ ]:
def chunk_text(text, max_tokens=510):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    for word in words:
        token_length = len(tokenizer.tokenize(word))
        if current_length + token_length > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(word)
        current_length += token_length
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

text_chunks = chunk_text(text)

In [ ]:
all_tokens = []
all_labels = []

In [ ]:
model.eval()
for chunk in text_chunks:
    # Add add_special_tokens=False
    tokens = tokenizer(chunk, return_offsets_mapping=True, return_tensors="pt", truncation=True, max_length=512, add_special_tokens=False)
    inputs = tokens["input_ids"]
    offsets = tokens.get("offset_mapping", [[]])[0]

    if inputs.shape[1] == 0 or len(offsets) == 0:
        continue  # Skip empty inputs

    with torch.no_grad():
        outputs = model(inputs).logits
    predictions = torch.argmax(outputs, dim=2).squeeze().tolist()

    for idx in range(len(offsets)):
        if idx >= len(predictions):
            continue  # Prevent index out of range
        start, end = offsets[idx]
        if start == 0 and end == 0:
            continue  # Skip special tokens
        if idx == 0 or (idx > 0 and offsets[idx - 1][1] == start):
            label = "B_WORD"
        elif idx + 1 < len(offsets) and offsets[idx + 1][0] == end:
            label = "I_WORD"
        else:
            label = "E_WORD"
        # Make sure token is in the tokenizer's vocabulary before converting
        if inputs[0][idx].item() in tokenizer.get_vocab():
            all_tokens.append(tokenizer.convert_ids_to_tokens(inputs[0][idx]))
            all_labels.append(label)
        else:
            # Handle unknown tokens (e.g., replace with [UNK])
            all_tokens.append(tokenizer.unk_token)
            all_labels.append(label)  # or assign a specific label for unknown tokens

In [ ]:
df = pd.DataFrame({"Token": all_tokens, "Label": all_labels})
df.to_csv("/content/ws_sample_submission.csv", index=False, encoding="utf-8")

print("Word segmentation completed. Output saved to ws_output.csv")

Word segmentation completed. Output saved to ws_output.csv
